In [38]:
import pandas as pd
import os
import json
if 'cd' not in globals():
    os.chdir('..')
    cd = True


In [40]:
with open('scripts/labels.json', 'r') as f:
    labels = json.load(f)

In [41]:
def check_characters(string, labels):
    for c in string:
        if c not in labels:
            return False
    return True

In [42]:
labels

['_',
 "'",
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 ' ']

In [46]:
speech_dir = 'speech/alaw'
trans_dir = 'transcriptions/ast/xsampa'

In [104]:
def extract_trans(strings, start=0.0):
#     print('extract trans')
#     print(strings, start)
    if not strings:
        return None
    words = []
    types = []
    for s in strings:
        end, _, ph, word, word_type = s.strip().split()
        if word_type != 'common' and word_type != '-':
#             print('error word type', word_type)
            return None
        if '-' in word:
            continue
        words.append(word.replace(';', ' ').upper())
    trans = ' '.join(words)
    return [trans, float(start), float(end)]

In [115]:
utt

'BE392LF012'

'BE392LF'

In [116]:

file_trans = []

for i, filename in enumerate(os.listdir(f'{speech_dir}')):
    if not filename.endswith('.wav'):
        continue

    if i % 1000 == 0:
        print(f'{i}th file')

    utt, extname = filename.split('.')
    spk = utt[:-3]
    trans_path = f'{trans_dir}/{utt}.t02'
    audio_path = f'{speech_dir}/{utt}.wav'
    with open(trans_path, 'r', encoding="ISO-8859-1") as f:
#         print(audio_path)
        started = False
        start_time = 0.0
        strings = []
        for line in f:
            if 'ETimes' in line:
#                 print('started')
                started = True
                continue

            if started:
                
                end, _, ph, word, word_type = line.strip().split()
#                 print(line)
                if '[' in word or ']' in word:
                    # end of a sentence
                    result = extract_trans(strings, start_time)
#                     print('end of a sentence', trans)
                    start_time = end
                    if result is not None:
                        trans, start, end = result
                        if end-start > 1.2 and len(trans) > 10 and check_characters(trans, labels):
                            file_trans.append([audio_path, trans, spk, start, end])
                    strings = []
                    continue
                    
                elif '-' in word:
                    strings.append(line.strip())
                    continue
                else:
                    strings.append(line.strip())
                    continue


0th file
3000th file
5000th file
6000th file
8000th file
9000th file
11000th file
12000th file
13000th file
15000th file
17000th file
18000th file


In [117]:
file_trans

[['speech/alaw/BE161CF036.wav',
  'ONE SIX THREE',
  'BE161CF',
  1.128585,
  2.333708],
 ['speech/alaw/BE161CF036.wav',
  'DOUBLE ONE DOUBLE ZERO',
  'BE161CF',
  2.45422,
  4.502928],
 ['speech/alaw/BE013CM026.wav',
  'THREE THREE SEVEN THREE',
  'BE013CM',
  0.430246,
  2.687463],
 ['speech/alaw/BE104LM031.wav',
  'ZERO NINE SIX',
  'BE104LM',
  0.472594,
  2.061412],
 ['speech/alaw/BE104LM031.wav',
  'ONE FIVE THREE',
  'BE104LM',
  2.20585,
  3.505793],
 ['speech/alaw/BE104LM031.wav',
  'SIX FIVE EIGHT FOUR',
  'BE104LM',
  3.794669,
  5.383486],
 ['speech/alaw/BE262LF037.wav',
  'GO FORWARD PLEASE',
  'BE262LF',
  0.660704,
  2.492972],
 ['speech/alaw/BE003LM007.wav', 'FIFTY EIGHT', 'BE003LM', 1.148753, 3.159397],
 ['speech/alaw/BE003LM007.wav',
  'ONE EIGHTY THREE',
  'BE003LM',
  10.950639,
  13.463947],
 ['speech/alaw/BE014CM015.wav',
  "IT'S TEN TWENTY FIVE",
  'BE014CM',
  4.025736,
  5.49967],
 ['speech/alaw/BE056CF021.wav',
  'BUT PRACTICALLY',
  'BE056CF',
  4.460569,
  5

In [111]:
len(file_trans)

3323

In [118]:
df = pd.DataFrame(file_trans, columns=['file', 'trans', 'speaker', 'start', 'end'])

In [121]:
n_speaker = len(df.speaker.unique())

In [123]:
all_speakers = sorted(df.speaker.unique())

In [124]:
n_speaker_train = int(n_speaker*0.8)
n_speaker_dev   = int(n_speaker*0.1)
n_speaker_test  = int(n_speaker*0.1)
train_speakers = all_speakers[:n_speaker_train]
dev_speakers   = all_speakers[n_speaker_train:n_speaker_train+n_speaker_dev]
test_speakers  = all_speakers[n_speaker_train+n_speaker_dev:]

In [126]:
len(train_speakers), len(dev_speakers), len(test_speakers)

(188, 23, 24)

In [129]:
df_train = df[df.speaker.isin(train_speakers)]
df_dev   = df[df.speaker.isin(dev_speakers)]
df_test  = df[df.speaker.isin(test_speakers)]

In [130]:
len(df_train), len(df_dev), len(df_test)

(2619, 335, 369)

In [131]:
df.to_csv('scripts/trans_all.csv')
df_train.to_csv('scripts/trans_train.csv')
df_dev.to_csv('scripts/trans_dev.csv')
df_test.to_csv('scripts/trans_test.csv')